# 🚀 MindMatrix Signed APK Builder (Kivy + KivyMD)
This Colab notebook will build a **signed APK** for your MindMatrix game.
Steps:
1. Install Buildozer & dependencies
2. Upload your `MindMatrix.zip`
3. Extract project files
4. Auto-generate a keystore
5. Configure `buildozer.spec`
6. Build signed APK
7. Download APK ✅

In [ ]:
# 🔹 Step 1: Install Buildozer & dependencies
!apt update
!apt install -y python3-pip build-essential git python3-dev ffmpeg \
libsdl2-dev libsdl2-image-dev libsdl2-mixer-dev libsdl2-ttf-dev libportmidi-dev \
libswscale-dev libavformat-dev libavcodec-dev zlib1g-dev unzip openjdk-17-jdk

!pip install --upgrade cython buildozer

In [ ]:
# 🔹 Step 2: Upload your MindMatrix.zip
from google.colab import files
uploaded = files.upload()   # Upload MindMatrix.zip here

In [ ]:
# 🔹 Step 3: Unzip & enter folder
import os, zipfile

zip_file = list(uploaded.keys())[0]   # Get uploaded filename
extract_folder = "MindMatrix"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

os.chdir(extract_folder)   # Go into MindMatrix folder
print("✅ Current folder:", os.getcwd())

In [ ]:
# 🔹 Step 4: Generate Keystore automatically
if not os.path.exists("mindmatrix.keystore"):
    !keytool -genkey -v -keystore mindmatrix.keystore -alias mindmatrix \
    -keyalg RSA -keysize 2048 -validity 10000 -storepass mindmatrix -keypass mindmatrix \
    -dname "CN=MindMatrix, OU=Dev, O=GameStudio, L=City, S=State, C=IN"
    
print("✅ Keystore generated: mindmatrix.keystore")

In [ ]:
# 🔹 Step 5: Create or update buildozer.spec
if not os.path.exists("buildozer.spec"):
    with open("buildozer.spec", "w") as f:
        f.write("""[app]
title = MindMatrix
package.name = mindmatrix
package.domain = org.example
source.include_exts = py,png,jpg,kv,atlas
requirements = python3,kivy,kivymd
orientation = portrait

[buildozer]
log_level = 2
warn_on_root = 0

[app:android]
android.release_keystore = mindmatrix.keystore
android.release_keyalias = mindmatrix
android.release_keystore_pass = mindmatrix
android.release_keyalias_pass = mindmatrix
""")
else:
    # Append signing config if missing
    with open("buildozer.spec", "a") as f:
        f.write("""
[app:android]
android.release_keystore = mindmatrix.keystore
android.release_keyalias = mindmatrix
android.release_keystore_pass = mindmatrix
android.release_keyalias_pass = mindmatrix
""")
print("✅ buildozer.spec is ready with signing config")

In [ ]:
# 🔹 Step 6: Build Signed APK
!buildozer -v android release

In [ ]:
# 🔹 Step 7: Download APK
import glob
from google.colab import files

apk_files = glob.glob("bin/*.apk")
if apk_files:
    print("✅ Signed APK generated:", apk_files[0])
    files.download(apk_files[0])
else:
    print("❌ No APK found. Check Buildozer output.")